## Part V: Training a Model with Data Programming

In [ ]:
# If necessary:
# import os
# os.remove('snorkel.db');
# os.system('cp snorkel.db\ features snorkel.db');

In [ ]:
%load_ext autoreload
%autoreload 2

from snorkel import SnorkelSession
session = SnorkelSession()

In [ ]:
from snorkel.models import candidate_subclass

Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

### Loading the `CandidateSet`, feature matrix, and label matrix

In [ ]:
from snorkel.models import CandidateSet
train = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates').one()

In [ ]:
from snorkel.annotations import FeatureManager

feature_manager = FeatureManager()

%time F_train = feature_manager.load(session, train, 'Train Features')

In [ ]:
from snorkel.annotations import LabelManager

label_manager = LabelManager()

%time L_train = label_manager.load(session, train, 'LF Labels')

## Train Generative Model

In [ ]:
from snorkel.learning import NaiveBayes

gen_model = NaiveBayes()
gen_model.train(L_train, n_iter=3000, rate=1e-5)

In [ ]:
gen_model.save(session, 'Generative Params')

In [ ]:
train_marginals = gen_model.marginals(L_train)

## Train Discriminative Model

In [ ]:
from snorkel.learning import LogReg

disc_model = LogReg()
disc_model.train(F_train, train_marginals, n_iter=5000, rate=1e-3)

In [ ]:
disc_model.w.shape

In [ ]:
%time disc_model.save(session, "Discriminative Params")

## Assess Performance on Dev Set

In [ ]:
from snorkel.models import CandidateSet
dev = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Development Candidates').one()

In [ ]:
%time F_dev = feature_manager.update(session, dev, 'Train Features', False)

In [ ]:
L_dev = label_manager.load(session, dev, "Hardware Development Labels -- Gold")

In [ ]:
gold_dev_set = session.query(CandidateSet).filter(CandidateSet.name == 'Hardware Development Candidates').one()

In [ ]:
F_dev.shape

In [ ]:
print len(F_dev.candidate_set)
print len(L_dev.candidate_set)
print len(F_dev.row_index)
print len(L_dev.row_index)

In [ ]:
tp, fp, tn, fn = disc_model.score(F_dev, L_dev, gold_dev_set)

In [ ]:
(len(tp) + len(fp) + len(tn) + len(fn))

In [ ]:
from hardware_utils import entity_level_f1

corpus = session.query(Corpus).filter(Corpus.name == 'Hardware Development').one()
entity_level_f1(tp, fp, tn, fn, filename, corpus, 'stg_temp_min')

In [ ]:
import os
os.system('cp snorkel.db snorkel.db\ final');

The End.